In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [ ]:
df_laptop = pd.read_csv('/content/drive/MyDrive/KLTN/Recomment_sytem/laptops_processed3.csv')

In [ ]:
# --- Bước 1: Mã hóa các cột phân loại (như bạn đã làm) ---
categorical_features = ['Brand', 'Generation', 'Core', 'Ram', 'SSD', 'Display', 'Graphics', 'OS', 'Warranty']
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_features = one_hot_encoder.fit_transform(df_laptop[categorical_features])
feature_names = one_hot_encoder.get_feature_names_out(categorical_features)
ohe_df = pd.DataFrame(ohe_features, columns=feature_names)

In [ ]:
# --- Bước 2: Chuẩn hóa cột giá tiền (`Price_VND`) ---
price_scaler = MinMaxScaler()
# Áp dụng scaler cho cột giá tiền. reshape(-1, 1) để chuyển Series thành 2D array
scaled_price = price_scaler.fit_transform(df_laptop[['Price_VND']])
scaled_price_df = pd.DataFrame(scaled_price, columns=['Price_VND_scaled'])


In [ ]:

# --- Bước 3: Kết hợp tất cả các đặc trưng vào một ma trận duy nhất ---
# Nối DataFrame đã được One-Hot Encoding với DataFrame giá đã chuẩn hóa
item_feature_matrix = pd.concat([ohe_df, scaled_price_df], axis=1)

In [ ]:
# In ra hình dạng của ma trận mới để kiểm tra
print("Hình dạng ma trận Item-Feature mới:", item_feature_matrix.shape)
print("\nMa trận Item-Feature mới (5 dòng đầu):")
print(item_feature_matrix.head())

Hình dạng ma trận Item-Feature mới: (914, 81)

Ma trận Item-Feature mới (5 dòng đầu):
   Brand_AXL  Brand_Acer  Brand_Apple  Brand_Asus  Brand_Avita  Brand_Chuwi  \
0        0.0         0.0          0.0         0.0          0.0          0.0   
1        0.0         0.0          0.0         0.0          0.0          0.0   
2        0.0         0.0          0.0         0.0          0.0          0.0   
3        0.0         0.0          0.0         0.0          0.0          0.0   
4        0.0         0.0          0.0         0.0          0.0          0.0   

   Brand_Colorful  Brand_Dell  Brand_Fujitsu  Brand_Gigabyte  ...  OS_DOS OS  \
0             0.0         0.0            0.0             0.0  ...        0.0   
1             0.0         0.0            0.0             0.0  ...        0.0   
2             0.0         0.0            0.0             0.0  ...        0.0   
3             0.0         0.0            0.0             0.0  ...        0.0   
4             0.0         0.0          

Với ma trận hồ sơ sản phẩm này, đã có mọi thứ cần để xây dựng hệ thống gợi ý. Bước tiếp theo là tính toán độ tương đồng (similarity) giữa tất cả các cặp laptop. Phương pháp phổ biến và hiệu quả nhất cho mục đích này là Cosine Similarity.

Công thức của Cosine Similarity tính toán góc giữa hai vector để xác định mức độ tương đồng của chúng. Giá trị trả về nằm trong khoảng từ 0 (không giống nhau) đến 1 (giống hệt nhau).

Để thực hiện, bạn có thể sử dụng hàm cosine_similarity từ thư viện sklearn.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Tính toán ma trận độ tương đồng Cosine
cosine_sim_matrix = cosine_similarity(item_feature_matrix)

In [ ]:
# Tạo DataFrame từ ma trận độ tương đồng
# df_laptop['Model'] được sử dụng làm chỉ mục (index) cho DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df_laptop['Model'], columns=df_laptop['Model'])

In [ ]:
# In ra 5 dòng và 5 cột đầu tiên của ma trận độ tương đồng để kiểm tra
print("Ma trận độ tương đồng Cosine (5x5):")
print(cosine_sim_df.iloc[:5, :5])

Ma trận độ tương đồng Cosine (5x5):
Model                                               HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11/ 4GB Graph)  \
Model                                                                                                                                           
HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen ...                                           1.000000                                           
Tecno Megabook T1 Laptop (11th Gen Core i3/ 8GB...                                           0.555590                                           
Lenovo V15 G4 ‎82YU00W7IN Laptop (AMD Ryzen 3 7...                                           0.555621                                           
Samsung Galaxy Book2 Pro 13 Laptop (12th Gen Co...                                           0.333868                                           
Xiaomi Redmi G Pro 2024 Gaming Laptop (14th Gen...                                           0

Getting Recommendation

In [ ]:
def get_recommendations_v2(model_name, cosine_sim_df, df_laptop, top_n=5):
    """
    Hàm này lấy gợi ý và trả về một DataFrame bao gồm cả điểm số tương đồng.
    """
    if model_name not in cosine_sim_df.index:
        return pd.DataFrame()

    sim_scores = cosine_sim_df.loc[model_name]
    sim_scores = sim_scores.sort_values(ascending=False)

    # Lấy top_n+1 gợi ý, bao gồm cả chính nó
    top_recommendations = sim_scores.iloc[:top_n + 1]

    # Lấy tên các model tương ứng
    recommended_models = top_recommendations.index

    # Lấy thông tin chi tiết của các model
    recommended_df = df_laptop[df_laptop['Model'].isin(recommended_models)].copy()

    # Thêm cột điểm số tương đồng
    recommended_df['Similarity Score'] = recommended_df['Model'].apply(
        lambda x: top_recommendations[x]
    )

    return recommended_df


In [ ]:

# Hàm hiển thị kết quả dễ nhìn hơn từ bước trước
def display_recommendations(recommended_df):
    if recommended_df.empty:
        print("Không có gợi ý nào để hiển thị.")
        return

    # Sắp xếp theo điểm tương đồng giảm dần
    recommended_df = recommended_df.sort_values(by='Similarity Score', ascending=False)

    # Chọn các cột quan trọng và đổi tên cho ngắn gọn
    display_df = recommended_df[['Model', 'Brand', 'Core', 'Ram', 'SSD', 'Graphics', 'Price_VND', 'Similarity Score']].copy()
    display_df.rename(columns={
        'Model': 'Laptop',
        'Price_VND': 'Giá (VND)',
        'Core': 'CPU',
        'Ram': 'RAM',
        'SSD': 'Ổ cứng',
        'Graphics': 'Card đồ họa',
        'Similarity Score': 'Độ tương đồng'
    }, inplace=True)

    # Định dạng cột giá tiền
    display_df['Giá (VND)'] = display_df['Giá (VND)'].apply(lambda x: f"{x:,.0f}".replace(",", "."))
    display_df['Độ tương đồng'] = display_df['Độ tương đồng'].apply(lambda x: f"{x:.2f}")

    # Hiển thị DataFrame mà không bị cắt
    print(display_df.to_string(index=False))

In [ ]:
# --- Thử nghiệm hàm với một sản phẩm bất kỳ ---
selected_model = 'HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11/ 4GB Graph)'
recommendations = get_recommendations_v2(selected_model, cosine_sim_df, df_laptop, top_n=5)

print(f"\nCác gợi ý tương tự cho '{selected_model}':")
display_recommendations(recommendations)


Các gợi ý tương tự cho 'HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11/ 4GB Graph)':
                                                                                    Laptop Brand               CPU            RAM Ổ cứng Card đồ họa  Giá (VND) Độ tương đồng
 HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11/ 4GB Graph)    HP 6-Core Mainstream  8 GB DDR4 RAM 512 GB  AMD Radeon 15.173.627          1.00
 HP Victus 15-fb0158AX Gaming Laptop (AMD Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11/ 4GB Graph)    HP 6-Core Mainstream  8 GB DDR4 RAM 512 GB  AMD Radeon 15.201.024          1.00
 HP Victus 15-fb0153AX Gaming Laptop (AMD Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11/ 4GB Graph)    HP 6-Core Mainstream  8 GB DDR4 RAM 512 GB  AMD Radeon 15.351.559          1.00
 HP Victus15-fb0147AX Gaming Laptop (Ryzen 5 5600H/ 8GB/ 512GB SSD/ Win11 Home/ 4GB Graph)    HP 6-Core Mainstream  8 GB DDR4 RAM 512 GB  AMD Radeon 14.749.419          1.00
 HP Victus 15

In [ ]:
selected_model_new = 'Samsung Galaxy Book2 Pro 13 Laptop (12th Gen Core i5/ 16GB/ 512GB SSD/ Win11)'
recommendations_new = get_recommendations_v2(selected_model_new, cosine_sim_df, df_laptop, top_n=20)
print(f"\nCác gợi ý tương tự cho '{selected_model_new}':")
display_recommendations(recommendations_new)


Các gợi ý tương tự cho 'Samsung Galaxy Book2 Pro 13 Laptop (12th Gen Core i5/ 16GB/ 512GB SSD/ Win11)':
                                                                                           Laptop   Brand               CPU            RAM Ổ cứng   Card đồ họa  Giá (VND) Độ tương đồng
                    Samsung Galaxy Book2 Pro 13 Laptop (12th Gen Core i5/ 16GB/ 512GB SSD/ Win11) Samsung  12-14 Core Power 16 GB DDR5 RAM 512 GB Intel Iris Xe 21.071.889          1.00
                Samsung Galaxy Book2 Pro 360 13 Laptop (12th Gen Core i5/ 16GB/ 512GB SSD/ Win11) Samsung  12-14 Core Power 16 GB DDR5 RAM 512 GB Intel Iris Xe 21.975.099          1.00
                             Dell Inspiron 5330 Laptop (13th Gen Core i5/ 16GB/ 512GB SSD/ Win11)    Dell  12-14 Core Power 16 GB DDR5 RAM 512 GB Intel Iris Xe 23.329.914          0.89
                             Dell Inspiron 5330 Laptop (13th Gen Core i7/ 16GB/ 512GB SSD/ Win11)    Dell  12-14 Core Power 16 GB DDR5 RAM 512 GB Intel Iri

In [ ]:
selected_model_old = 'Tecno Megabook T1 Laptop (11th Gen Core i3/ 8GB/ 512GB SSD/ Win11 Home)'
recommendations_old = get_recommendations_v2(selected_model_old, cosine_sim_df, df_laptop, top_n=20)
print(f"\nCác gợi ý tương tự cho '{selected_model_old}':")
display_recommendations(recommendations_old)


Các gợi ý tương tự cho 'Tecno Megabook T1 Laptop (11th Gen Core i3/ 8GB/ 512GB SSD/ Win11 Home)':
                                                                                     Laptop   Brand          CPU           RAM Ổ cứng Card đồ họa  Giá (VND) Độ tương đồng
                    Tecno Megabook T1 Laptop (11th Gen Core i3/ 8GB/ 512GB SSD/ Win11 Home)   Tecno 2-Core Entry 8 GB DDR4 RAM 512 GB   Intel UHD  7.222.669          1.00
Asus Vivobook 15 Touch X515EA-EZ311WS Laptop (11th Gen Core i3/ 8GB/ 512GB SSD/ Win11 Home)    Asus 2-Core Entry 8 GB DDR4 RAM 512 GB   Intel UHD  9.631.229          0.89
          Asus VivoBook 15 X1500EA-EJ326WS Laptop (11th Gen Core i3/ 8GB/ 512GB SSD/ Win11)    Asus 2-Core Entry 8 GB DDR4 RAM 512 GB   Intel UHD  9.631.229          0.89
     Acer Aspire Lite AL15-51 UN.431SI.252 Laptop (11th Gen Core i3/ 8GB/ 512GB SSD/ Win11)    Acer 2-Core Entry 8 GB DDR4 RAM 512 GB   Intel UHD  9.751.657          0.89
                          Dell Vostro 3520 Lap

In [ ]:
# Xác định đường dẫn để lưu tệp
output_file_path = '/content/drive/MyDrive/KLTN/Recomment_sytem/cosine_similarity_matrix.csv'

# Lưu DataFrame ma trận độ tương đồng ra tệp CSV
cosine_sim_df.to_csv(output_file_path)

print(f"Đã lưu ma trận độ tương đồng tại: {output_file_path}")

Đã lưu ma trận độ tương đồng tại: /content/drive/MyDrive/KLTN/Recomment_sytem/cosine_similarity_matrix.csv


In [ ]:
import pickle

# Xác định đường dẫn để lưu tệp
output_file_path_pickle = '/content/drive/MyDrive/KLTN/Recomment_sytem/cosine_similarity_matrix.pkl'

# Lưu DataFrame ma trận độ tương đồng ra tệp pickle
with open(output_file_path_pickle, 'wb') as f:
    pickle.dump(cosine_sim_df, f)

print(f"Đã lưu ma trận độ tương đồng dưới dạng pickle tại: {output_file_path_pickle}")

Đã lưu ma trận độ tương đồng dưới dạng pickle tại: /content/drive/MyDrive/KLTN/Recomment_sytem/cosine_similarity_matrix.pkl
